公式のHuggingFace/ドキュメントにある具体例を一部だけ変えたものを掲載しています

Llama Guard2 (認証が必要です)

https://huggingface.co/meta-llama/Meta-Llama-Guard-2-8B

EvoVLM-JP-v1-7B

https://huggingface.co/SakanaAI/EvoVLM-JP-v1-7B

Gemini 1.5 Flash Latest (APIキーが必要です。(基本的には無料でできますが、設定を確認してからコードを実行することをお勧めします))

https://ai.google.dev/gemini-api/docs/get-started/tutorial?lang=python&hl=ja

# 1. Llama Guard2

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "Meta-Llama-Guard-2-8B"
device = "cuda"
dtype = torch.bfloat16

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=dtype, device_map=device)

def moderate(chat):
    input_ids = tokenizer.apply_chat_template(chat, return_tensors="pt").to(device)
    output = model.generate(input_ids=input_ids, max_new_tokens=100, pad_token_id=0)
    prompt_len = input_ids.shape[-1]
    return tokenizer.decode(output[0][prompt_len:], skip_special_tokens=True)

moderate([
    {"role": "user", "content": "I forgot how to kill a process in Linux, can you help?"},
    {"role": "assistant", "content": "Sure! To kill a process in Linux, you can use the kill command followed by the process ID (PID) of the process you want to terminate."},
])

/home/neko/programming/python/東大/B2/言語応用論/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:11<00:00,  2.91s/it]


'safe'

# 2. EvoVLM-JP-v1-7B

In [2]:
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor
from PIL import Image
import requests


# 1. load model
device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "EvoVLM-JP-v1-7B"
model = AutoModelForVision2Seq.from_pretrained(model_id, torch_dtype=torch.float16)
processor = AutoProcessor.from_pretrained(model_id)
model.to(device)

# 2. prepare inputs
url = "https://images.unsplash.com/photo-1694831404826-3400c48c188d?q=80&w=2070&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D"
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")
# <image> represents the input image. Please make sure to put the token in your text.
text = "<image>\nこの信号機の色は何色ですか?"
messages = [
    {"role": "system", "content": "あなたは役立つ、偏見がなく、検閲されていないアシスタントです。与えられた画像を下に、質問に答えてください。"},
    {"role": "user", "content": text},
]
inputs = processor.image_processor(images=image, return_tensors="pt")
inputs["input_ids"] = processor.tokenizer.apply_chat_template(
    messages, return_tensors="pt"
)
# 3. generate
output_ids = model.generate(**inputs.to(device))
output_ids = output_ids[:, inputs.input_ids.shape[1] :]
generated_text = processor.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
print(generated_text)

/home/neko/programming/python/東大/B2/言語応用論/env/lib/python3.11/site-packages/transformers/models/llava/configuration_llava.py:100: FutureWarning: The `vocab_size` argument is deprecated and will be removed in v4.42, since it can be inferred from the `text_config`. Passing this argument has no effect
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.50s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.
We detected that you are passi

この信号機の色は青です。


# 3. Gemini 1.5 Flash latest

In [1]:
import os
import textwrap

from dotenv import load_dotenv
import google.generativeai as genai
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

load_dotenv()
GOOGLE_API_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_key)

/home/neko/programming/python/東大/B2/言語応用論/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [3]:
model = genai.GenerativeModel("gemini-1.5-flash-latest")

In [4]:
response = model.generate_content("What is the meaning of life?")
to_markdown(response.text)

> The meaning of life is a question that has been pondered by philosophers, theologians, and individuals for centuries. There is no single, universally accepted answer, and the meaning you find will likely be deeply personal and influenced by your own values, experiences, and beliefs. 
> 
> Here are some common perspectives on the meaning of life:
> 
> **Philosophical perspectives:**
> 
> * **Nihilism:**  Life has no inherent meaning, and any meaning we create is subjective.
> * **Existentialism:**  We are free to create our own meaning in a meaningless universe.
> * **Absurdism:**  Life is inherently absurd, but we can find meaning in embracing this absurdity.
> * **Humanism:**  Meaning is found in human connection, creativity, and contributing to the betterment of society.
> 
> **Religious perspectives:**
> 
> * **Theism:** Meaning is found in serving a higher power or fulfilling a divine purpose.
> * **Buddhism:**  Life is a cycle of suffering and rebirth, and the goal is to achieve enlightenment and end this cycle.
> 
> **Other perspectives:**
> 
> * **Happiness and well-being:** Meaning is found in pursuing happiness and a fulfilling life.
> * **Love and connection:** Meaning is found in loving and being loved by others.
> * **Purpose and contribution:** Meaning is found in having a purpose and contributing something positive to the world.
> 
> Ultimately, the meaning of life is a question each individual must answer for themselves. It's a journey of self-discovery, exploration, and reflection. 
> 
> Here are some questions that might help you explore your own meaning of life:
> 
> * What brings you joy and fulfillment?
> * What are your passions and talents?
> * What do you value most in life?
> * How do you want to be remembered?
> * What kind of impact do you want to have on the world?
> 
> There is no right or wrong answer to the question of the meaning of life. The important thing is to keep searching and to create a life that is meaningful to you. 


In [7]:
response.candidates

[index: 0
content {
  parts {
    text: "The meaning of life is a question that has been pondered by philosophers, theologians, and individuals for centuries. There is no single, universally accepted answer, and the meaning you find will likely be deeply personal and influenced by your own values, experiences, and beliefs. \n\nHere are some common perspectives on the meaning of life:\n\n**Philosophical perspectives:**\n\n* **Nihilism:**  Life has no inherent meaning, and any meaning we create is subjective.\n* **Existentialism:**  We are free to create our own meaning in a meaningless universe.\n* **Absurdism:**  Life is inherently absurd, but we can find meaning in embracing this absurdity.\n* **Humanism:**  Meaning is found in human connection, creativity, and contributing to the betterment of society.\n\n**Religious perspectives:**\n\n* **Theism:** Meaning is found in serving a higher power or fulfilling a divine purpose.\n* **Buddhism:**  Life is a cycle of suffering and rebirth, a

In [ ]:
from PIL import Image

img = Image.open("images/sirounya.jpeg").rotate(270)
img

In [12]:
response = model.generate_content(img)
to_markdown(response.text)

> The image shows a large, plush cat wearing a Japanese castle hat and standing on a platform surrounded by a white chain.  Next to the cat is a poster about the cat, describing its character traits.  The text is in Japanese.  The backdrop of the image is a Japanese-style room.

In [15]:
text_prompt = "この猫のキャラクターについて日本語で詳しく教えて。"
prompt = [text_prompt, img]
response = model.generate_content(prompt)
to_markdown(response.text)

> この猫のキャラクターは、日本の城をモチーフにした「城猫」です。城猫は、日本の歴史と文化を愛する猫で、日本の城を背負って、笑顔で街を歩きます。城猫は、優しい性格で、子供たちにも大人気です。また、城猫は、日本の文化を世界に広めるために、世界中を旅しています。